# FIT5202 Assignment 2A : Building Models for eCommerce Fraud Detection

## Table of Contents
*  
    * [Part 1 : Data Loading, Transformation and Exploration](#part-1)
    * [Part 2 : Feature extraction and ML training](#part-2)
    * [Part 3 : Customer Segmentation and Knowledge sharing with K-Mean](#part-3)
    * [Part 4 : Data Ethics, Privacy, and Security](#part-4)
 
Please add code/markdown cells if needed.

# Part 1: Data Loading, Transformation and Exploration <a class="anchor" name="part-1"></a>
## 1.1 Data Loading
In this section, you must load the given datasets into PySpark DataFrames and use DataFrame functions to process the data. Spark SQL usage is discouraged, and you can only use pandas to format results. For plotting, various visualisation packages can be used, but please ensure that you have included instructions to install the additional packages and that the installation will be successful in the provided docker container (in case your marker needs to clear the notebook and rerun it).

### 1.1.1 Data Loading <a class="anchor" name="1.1"></a>
1.1.1 Write the code to create a SparkSession. For creating the SparkSession, you need to use a SparkConf object to configure the Spark app with a proper application name, to ensure the maximum partition size does not exceed 16MB, and to run locally with all CPU cores on your machine (note: if you have insufficient RAM, reducing the number of cores is acceptable.)  (2%)

In [1]:
from pyspark import SparkConf
master = "local[*]"
app_name = "FIT5202 A2A"
spark_conf = SparkConf().setMaster(master).setAppName(app_name)
from pyspark import SparkContext
from pyspark.sql import SparkSession 
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

1.1.2 Write code to define the schemas for the category, customer, product, browsing behaviour and transaction datasets, following the data types suggested in the metadata file. (3%)

In [2]:
from pyspark.sql.types import *

category_schema = StructType([ 
    StructField("category_id", IntegerType(), True), 
    StructField("cat_level1", StringType(), True), 
    StructField("cat_level2", StringType(), True),
    StructField("cat_level3", StringType(), True)
])

customer_schema = StructType([ 
    StructField("customer_id", IntegerType(), True), 
    StructField("first_name", StringType(), True), 
    StructField("last_name", StringType(), True),
    StructField("username", StringType(), True),
    StructField("email", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("birthdate", DateType(), True),
    StructField("first_join_date", DateType(), True)
])

product_schema = StructType([ 
    StructField("id", IntegerType(), True), 
    StructField("gender", StringType(), True), 
    StructField("baseColour", StringType(), True),
    StructField("season", StringType(), True),
    StructField("year", IntegerType(), True),
    StructField("usage", StringType(), True),
    StructField("productDisplayName", StringType(), True),
    StructField("category_id", IntegerType(), True)
])

browsing_behaviour_schema = StructType([ 
    StructField("session_id", StringType(), True), 
    StructField("event_type", StringType(), True), 
    StructField("event_time", TimestampType(), True),
    StructField("traffic_source", StringType(), True),
    StructField("device_type", StringType(), True)
])

transaction_schema = StructType([
    StructField("created_at", TimestampType(), True),
    StructField("customer_id", IntegerType(), True),  
    StructField("transaction_id", StringType(), True),  
    StructField("session_id", StringType(), True),  
    StructField("product_metadata", StringType(), True),
    StructField("payment_method", StringType(), True),
    StructField("payment_status", StringType(), True),
    StructField("promo_amount", DoubleType(), True),
    StructField("promo_code", StringType(), True),
    StructField("shipment_fee", DoubleType(), True),
    StructField("shipment_location_lat", DoubleType(), True),
    StructField("shipment_location_long", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("clear_payment", IntegerType(), True)
])

1.1.3 Using predefined schemas, write code to load the CSV files into separate data frames. Print the schemas of all data frames. (2%)

In [3]:
df_category = spark.read.format('csv')\
    .option('header', True).option('escape', '"')\
    .schema(category_schema)\
    .load('category.csv')

df_customers = spark.read.format('csv')\
    .option('header', True).option('escape', '"')\
    .schema(customer_schema)\
    .load('customer.csv')

df_transactions = spark.read.format('csv')\
    .option('header', True).option('escape', '"')\
    .schema(transaction_schema)\
    .load('transactions.csv')

df_product = spark.read.format('csv')\
    .option('header', True).option('escape', '"')\
    .schema(product_schema)\
    .load('product.csv')

df_browsing_behaviour = spark.read.format('csv')\
    .option('header', True).option('escape', '"')\
    .schema(browsing_behaviour_schema)\
    .load('browsing_behaviour.csv')

df_customer_session = spark.read.format('csv')\
    .option('header', True).option('escape', '"')\
    .load('customer_session.csv')

df_fraud_transactions = spark.read.format('csv')\
    .option('header', True).option('escape', '"')\
    .load('fraud_transaction.csv')


In [27]:
df_category.printSchema()
df_customers.printSchema()
df_transactions.printSchema()
df_product.printSchema()
df_browsing_behaviour.printSchema()
df_customer_session.printSchema()
df_fraud_transactions.printSchema()

root
 |-- category_id: integer (nullable = true)
 |-- cat_level1: string (nullable = true)
 |-- cat_level2: string (nullable = true)
 |-- cat_level3: string (nullable = true)

root
 |-- customer_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- username: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- first_join_date: date (nullable = true)

root
 |-- created_at: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- transaction_id: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- product_metadata: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- payment_status: string (nullable = true)
 |-- promo_amount: double (nullable = true)
 |-- promo_code: string (nullable = true)
 |-- shipment_fee: double (nullable = true)
 |-- shipment_location_lat: double (null

### 1.2 Data Transformation to Create Features <a class="anchor" name="1.2"></a>
In the browsing behaviour dataset, there are 10 types of events:  
VC(Viewing Category), VI(Viewing Item), VP(Viewing Promotion), AP(Add Promotion), CL(Click on a product/category) , ATC(Add a product to Shopping Cart), CO(CheckOut), HP(View HomePage), SCR(Mouse Scrolling), SER(Search for a product/category)  
We categorise them into three different levels:  
L1(actions that are highly likely lead to a purchase): AP, ATC, CO  
L2(actions may lead to purchase): VC, VP, VI, SER  
L3(not very important - just browsing):  SCR, HP, CL  
Perform the following tasks based on the loaded data frames and create a new data frame.  

1.2.1 For each transaction (linked to a browsing session), count the number of actions in each level and create 3 columns(L1_count, L2_count, L3_count).

In [40]:
from pyspark.sql import functions as F

# first join transaction and browsing session with 'session_id'. 
df_transbrows_session = df_transactions.join(df_browsing_behaviour, on="session_id", how="left")
# df_transbrows_session.select("transaction_id", "session_id", "event_type").show()

L1_events = ['AP', 'ATC', 'CO']
L2_events = ['VC', 'VP', 'VI', 'SER']
L3_events = ['SCR', 'HP', 'CL']

df_transbrows_session = df_transbrows_session.withColumn(
    "event_level", 
    F.when(F.col("event_type").isin(L1_events), "L1")
     .when(F.col("event_type").isin(L2_events), "L2")
     .when(F.col("event_type").isin(L3_events), "L3")
     .otherwise("Unknown")
)
df_transbrows_session_agg = df_transbrows_session.groupBy("transaction_id").agg(
    F.sum(F.when(F.col("event_level") == "L1", 1).otherwise(0)).alias("L1_count"),
    F.sum(F.when(F.col("event_level") == "L2", 1).otherwise(0)).alias("L2_count"),
    F.sum(F.when(F.col("event_level") == "L3", 1).otherwise(0)).alias("L3_count")
)
df_transbrows_session_agg.show(truncate=False)

+------------------------------------+--------+--------+--------+
|transaction_id                      |L1_count|L2_count|L3_count|
+------------------------------------+--------+--------+--------+
|6211f29a-8435-4e22-a56c-0d8e5a114e48|7       |4       |9       |
|37280714-c8e9-45d0-b089-b161435d27df|3       |11      |19      |
|763e6b0a-faa6-4814-9ec6-a2cdfdd2ddc1|2       |3       |4       |
|1b412535-32c7-4db4-97b2-67a6f5f56b6e|9       |8       |12      |
|9144d238-525a-4833-87af-7e92a6ce8eaf|4       |8       |16      |
|303c188e-3cfc-4990-bcf6-9c28ec92834f|5       |0       |2       |
|74dce22c-d1ef-4f18-a4bb-fc4e658ef178|4       |1       |2       |
|57de19b5-ee84-4726-88a8-07bca4d0b8b3|3       |1       |4       |
|2755eefe-f1fb-4f23-a6aa-a7e53f573311|7       |6       |5       |
|f3c01706-2016-4614-9164-3675f35ad75c|3       |0       |2       |
|c5dc7f18-0276-41f9-aa02-50d1503bdf51|3       |2       |5       |
|3a282187-656e-4ed2-9922-a4410159df08|2       |45      |44      |
|498446e5-

In [42]:
# checking if aggregation done right by using sample transaction id
df_test = df_transbrows_session.filter(F.col("transaction_id") == "6211f29a-8435-4e22-a56c-0d8e5a114e48")
df_test.select("transaction_id", "event_type", "event_level").show(truncate=False)

+------------------------------------+----------+-----------+
|transaction_id                      |event_type|event_level|
+------------------------------------+----------+-----------+
|6211f29a-8435-4e22-a56c-0d8e5a114e48|HP        |L3         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|ATC       |L1         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|HP        |L3         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|CL        |L3         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|CL        |L3         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|ATC       |L1         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|ATC       |L1         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|ATC       |L1         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|SCR       |L3         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|HP        |L3         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|ATC       |L1         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|HP        |L3         |
|6211f29a-8435-4e22-a56c-0d8e5a114e48|ATC       |L1         |
|6211f29

1.2.2 Create two columns with a percentage ratio of L1 and L2 actions. (i.e. L1 ratio = L1/(L1+L2+L3) * 100%)

L1 ratio = L1/(L1+L2+L3) * 100% \
L2 ratio = L2/(L1+L2+L3) * 100%

In [44]:
df_event_ratio = df_transbrows_session_agg.withColumn("L1_ratio", (F.col("L1_count") / (F.col("L1_count") + F.col("L2_count") + F.col("L3_count"))) * 100
).withColumn( "L2_ratio", (F.col("L2_count") / (F.col("L1_count") + F.col("L2_count") + F.col("L3_count"))) * 100
)

df_with_ratios.select("transaction_id", "L1_count", "L2_count", "L3_count", "L1_ratio", "L2_ratio").show()

+--------------------+--------+--------+--------+------------------+------------------+
|      transaction_id|L1_count|L2_count|L3_count|          L1_ratio|          L2_ratio|
+--------------------+--------+--------+--------+------------------+------------------+
|6211f29a-8435-4e2...|       7|       4|       9|              35.0|              20.0|
|37280714-c8e9-45d...|       3|      11|      19| 9.090909090909092| 33.33333333333333|
|763e6b0a-faa6-481...|       2|       3|       4| 22.22222222222222| 33.33333333333333|
|1b412535-32c7-4db...|       9|       8|      12| 31.03448275862069|27.586206896551722|
|9144d238-525a-483...|       4|       8|      16|14.285714285714285| 28.57142857142857|
|303c188e-3cfc-499...|       5|       0|       2| 71.42857142857143|               0.0|
|74dce22c-d1ef-4f1...|       4|       1|       2| 57.14285714285714|14.285714285714285|
|57de19b5-ee84-472...|       3|       1|       4|              37.5|              12.5|
|2755eefe-f1fb-4f2...|       7| 

1.2.3 For each unique browsing session, based on event_time, extract the time of day as 4 groups: morning(6am-11:59am), afternoon(12pm-5:59pm), evening(6pm-11:59pm), night(12am-5:59am), add a column. (note: use medium time if a browsing session spans across different groups. For example, if a session starts at 10 am and ends at 1 pm, use 11:30 => (10+13)/2).

In [ ]:
df_browsing_session

def extract_time(event_time):
    


In [62]:
df_test = df_browsing_behaviour.filter(F.col("session_id") == "c9718135-8134-42b2-8e1e-2737fd6b49b1")
df_test.select("session_id", "event_time").orderBy("event_time").show(truncate=False)

+------------------------------------+-----------------------+
|session_id                          |event_time             |
+------------------------------------+-----------------------+
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2020-11-14 13:02:39.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2020-11-25 11:01:01.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2020-11-25 11:17:01.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2020-12-06 09:07:47.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2020-12-06 09:12:13.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2020-12-17 07:04:52.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2020-12-17 07:12:08.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2020-12-28 04:49:04.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2021-01-08 02:28:32.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2021-01-19 00:17:38.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2021-01-29 22:34:04.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2021-02-09 20:00:24.331|
|c9718135-8134-42b2-8e1e-2737fd6b49b1|2021-03-25 11:43:

In [61]:
from pyspark.sql import functions as F

# Step 1: Extract the date from the event_time
df_transbrows_session = df_transbrows_session.withColumn("event_date", F.to_date(F.col("event_time")))

# Step 2: Group by session_id and event_date, and calculate min and max event_time for each date
df_session_time = df_transbrows_session.groupBy("session_id", "event_date").agg(
    F.min("event_time").alias("min_time"),
    F.max("event_time").alias("max_time")
)

# Step 3: Filter for a specific session to check results
df_test = df_session_time.filter(F.col("session_id") == "d31b9d4b-126a-49c4-be4b-8e5d7f70804d")

# Show the result
df_test.show(truncate=False)


+------------------------------------+----------+-----------------------+-----------------------+
|session_id                          |event_date|min_time               |max_time               |
+------------------------------------+----------+-----------------------+-----------------------+
|d31b9d4b-126a-49c4-be4b-8e5d7f70804d|2020-04-25|2020-04-25 04:32:54.612|2020-04-25 04:36:02.612|
|d31b9d4b-126a-49c4-be4b-8e5d7f70804d|2020-05-10|2020-05-10 07:53:43.612|2020-05-10 07:53:43.612|
|d31b9d4b-126a-49c4-be4b-8e5d7f70804d|2020-05-06|2020-05-06 00:01:57.612|2020-05-06 00:01:57.612|
|d31b9d4b-126a-49c4-be4b-8e5d7f70804d|2020-05-07|2020-05-07 01:52:38.612|2020-05-07 01:52:38.612|
|d31b9d4b-126a-49c4-be4b-8e5d7f70804d|2020-05-04|2020-05-04 22:04:00.612|2020-05-04 22:04:00.612|
|d31b9d4b-126a-49c4-be4b-8e5d7f70804d|2020-05-03|2020-05-03 19:56:51.612|2020-05-03 19:56:51.612|
|d31b9d4b-126a-49c4-be4b-8e5d7f70804d|2020-05-02|2020-05-02 17:53:16.612|2020-05-02 17:53:16.612|
|d31b9d4b-126a-49c4-

In [50]:
df_browsing_behaviour.show(truncate=False)

+------------------------------------+----------+-----------------------+--------------+-----------+
|session_id                          |event_type|event_time             |traffic_source|device_type|
+------------------------------------+----------+-----------------------+--------------+-----------+
|c9718135-8134-42b2-8e1e-2737fd6b49b1|AP        |2020-12-17 07:04:52.331|MOBILE        |Android    |
|c9718135-8134-42b2-8e1e-2737fd6b49b1|CL        |2020-12-06 09:07:47.331|MOBILE        |Android    |
|c9718135-8134-42b2-8e1e-2737fd6b49b1|SER       |2020-12-17 07:12:08.331|MOBILE        |Android    |
|c9718135-8134-42b2-8e1e-2737fd6b49b1|CL        |2021-01-08 02:28:32.331|MOBILE        |Android    |
|c9718135-8134-42b2-8e1e-2737fd6b49b1|ATC       |2021-01-19 00:17:38.331|MOBILE        |Android    |
|c9718135-8134-42b2-8e1e-2737fd6b49b1|VI        |2020-12-28 04:49:04.331|MOBILE        |Android    |
|d2077615-9026-4c5c-b088-a35519b64b05|HP        |2020-02-23 23:03:14.612|MOBILE        |And

1.2.4 Join data frames to find customer information and add columns to feature_df: gender, age, geolocation, first join year. (note: For some columns, you need to perform transformations. For age, keep the integer only by rounding.)

In [4]:
# determine geolocation using shipment long lat
df_geolocation = df_transactions.select("customer_id", "shipment_location_lat", "shipment_location_long")
df_geolocation.show(truncate=False)

+-----------+---------------------+----------------------+
|customer_id|shipment_location_lat|shipment_location_long|
+-----------+---------------------+----------------------+
|14159      |-4.26351275671241    |105.489401701251      |
|22576      |-7.91707661186231    |110.13187555325       |
|18696      |-7.39661418330981    |109.511262594032      |
|90136      |-0.637290541399757   |109.492521253314      |
|18960      |-7.32004136393024    |111.225797135699      |
|60646      |-4.52328589944563    |105.385799510518      |
|5901       |-7.43210236666926    |111.096960686913      |
|69072      |-6.26355191799179    |106.859716713089      |
|92076      |-0.420556955342107   |113.93175424743       |
|51799      |-2.98365414329573    |101.93104883237       |
|32840      |-6.29321554243872    |106.891616710191      |
|93365      |3.11080990817827     |99.5849034250288      |
|6664       |-5.32866351581116    |105.431488069038      |
|4343       |-6.35528560922436    |106.909932739596     

In [12]:
location = geolocator.reverse("-7.91707661186231, 110.13187555325 ")
print(location.address)

Garongan, Panjatan, Kulon Progo, Daerah Istimewa Yogyakarta, Jawa, Indonesia


In [27]:
# pip install geopy - find customer address base don their shipment latlong
from geopy.geocoders import Nominatim
from pyspark.sql.functions import udf

def find_address(shipment_location_lat, shipment_location_long):
    geolocator = Nominatim(user_agent="findAddress")
    location = geolocator.reverse((shipment_location_lat, shipment_location_long))
    return location.address if location else None  

address_udf = udf(find_address,StringType())
df_geolocation2 = df_geolocation.withColumn('address', address_udf('shipment_location_lat', 'shipment_location_long'))

In [28]:
df_geolocation2.show(truncate=False)

+-----------+---------------------+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|customer_id|shipment_location_lat|shipment_location_long|address                                                                                                                                                   |
+-----------+---------------------+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|14159      |-4.26351275671241    |105.489401701251      |Marga Jaya, Tulang Bawang, Lampung, Sumatera, Indonesia                                                                                                   |
|22576      |-7.91707661186231    |110.13187555325       |Garongan, Panjatan, Kulon Progo, Daerah Istimewa Yogyakarta, Jawa, Indonesia          

1.2.5 Join data frames to find out the number of purchases the customer has made, add a column.

1.2.6 Attach the transaction labels for fraud/non-fraud.

### 1.3 Exploring the Data <a class="anchor" name="1.3"></a>
**1.3.1 With the feature_df, write code to show the basic statistics: a) For each numeric column, show count, mean, stddev, min, max, 25 percentile, 50 percentile, 75 percentile; b) For each non-numeric column, display the top-5 values and the corresponding counts; c) For each boolean column, display the value and count. (3%)**

**1.3.2 Explore the dataframe and write code to present two plots worthy of presentation to the company, describe your plots and discuss the findings from the plots. (8%)**
One of the plots needs to be based on feature_df in regard to fraudulent behaviour; you’re free to choose the other one.  
Hint 1: You can use basic plots (e.g., histograms, line charts, scatter plots) to show the relationship between a column and the label or more advanced plots like correlation plots.  
Hint 2: If your data is too large for plotting, consider using sampling before plotting.  
150 words max for each plot’s description and discussion  
Feel free to use any plotting libraries: matplotlib, seabon, plotly, etc.  



## Part 2. Feature extraction and ML training <a class="anchor" name="part-2"></a>
In this section, you must use PySpark DataFrame functions and ML packages for data preparation, model building, and evaluation. Other ML packages, such as scikit-learn, would receive zero marks.
### 2.1 Discuss the feature selection and prepare the feature columns

2.1.1 Based on the data exploration from 1.2 and considering the use case, discuss the importance of those features (For example, which features may be useless and should be removed, which feature has a significant impact on the label column, which should be transformed), which features you are planning to use? Discuss the reasons for selecting them and how you create/transform them
300 words max for the discussion
Please only use the provided data for model building
You can create/add additional features based on the dataset
Hint - Use the insights from the data exploration/domain knowledge/statistical models to consider whether to create more feature columns, whether to remove some columns

2.1.2 Write code to create/transform the columns based on your discussion above
Hint: You can use one data frame for both use cases (classification and k-mean later in part 3) since you can select your desired columns as the input and output for each use case. 

### 2.2 Preparing Spark ML Transformers/Estimators for features, labels, and models  <a class="anchor" name="2.2"></a>

**2.2.1 Write code to create Transformers/Estimators for transforming/assembling the columns you selected above in 2.1 and create ML model Estimators for Random Forest (RF) and Gradient-boosted tree (GBT) model.
Please DO NOT fit/transform the data yet.**

**2.2.2. Write code to include the above Transformers/Estimators into two pipelines.
Please DO NOT fit/transform the data yet.**

### 2.3 Preparing the training data and testing data  
Write code to split the data for training and testing purposes.
Note: Due to the large dataset size, you can use random sampling (say 20% of the dataset) and do a train/test split or use one year of data for training and another year for testing. 

### 2.4 Training and evaluating models  
2.4.1 Write code to use the corresponding ML Pipelines to train the models on the training data from 2.3. And then use the trained models to predict the testing data from 2.3

2.4.2 For both models (RF and GBT) and testing data, write code to display the count of TP/TN/FP/FN. Compute the AUC, accuracy, recall, and precision for the above-threshold/below-threshold label from each model testing result using PySpark MLlib/ML APIs.
Draw a ROC plot.
Discuss which one is the better model (no word limit; please keep it concise)

2.4.3 Save the better model (you need it for Part B of Assignment 2).
(Note: You may need to go through a few training loops or use more data to create a better-performing model.)

### Part 3. Customer Clustering and Knowledge sharing with K-Mean <a class="anchor" name="part-3"></a>  
Please see the specification for this task and add code/markdown cells.

### Part 4: Data Ethics, Privacy, and Security <a class="anchor" name="part-4"></a>  
Please see the specification for this task and add markdown cells(word limit: 500).

## References:
Please add your references below: